<a href="https://colab.research.google.com/github/realBagher/fundemental_Ai_algorithms/blob/main/genetic_algorithms/genetic_algorithmV2.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

## implementation of Gentic Algorithm from Scratch using numpy and pandas 




In [ ]:
import pandas as pd
import numpy as np
import matplotlib.pyplot as plt 
import math
from tqdm import tqdm 

**Helper functions **

**Sigmoid Function & tanh Functions**
> in my initial approach, using *sigmoid* or *hyperbolic* functions seemed probable for normalizing the fitness value of each individual however as we moved onwards we faced significant draw-backs as the range of the fitness values are squashed and *SGA* lacks the ability to work with such ranges of numbers and this has a significant impact on our parent selection mechanism  

In [ ]:
def sigmoid(z):
  return 1/(1 + np.exp(-z))

In [ ]:
def tanh(z):
  return (np.exp(z)-np.exp(-z))/(np.exp(z)+np.exp(-z))

**Fitness Evaluation Function**
> further analyzing the given Quadratic we can deduce that we are solving for a maximum as the curvature of the function is upside-down thus we can use the value of the actual function as the diffrentiating measure which is define as *func* 

In [ ]:
def func(x):
  return ((-x**2)+6*x)

**INITIALIZING GENERATIONS**
> using *np.random.randint* we intialize k units of individuals in range of 0 to 30  

In [ ]:
def intialize(k):
  population = np.random.randint(0,30,k) 
  return population

**Evalutation Function**
> we simply map every individual to the *func* and return the value as the fitness of every individual 

In [ ]:
def evaluation(x):
  value = []
  
  for i in x:
    value.append(func(i))

  
  return value
  # return func(x)
  # return sigmoid(func(x))
  # return tanh(func(x))

**PROBABILITY GENERATION FOR RUSSIAN ROULETE**


*   we should note that the sum of the probability is equal to 1
*   we should also note that the probability of each indiviual can't be a negative value 
* and even the worst indiviual must have a chance 
> considering the above conditions we set out to add the **minimum fitness** of the generation + 1 to each so that all probabilities remain positive and comparable
<br>
> **+1** is to give the worst a guy a chace however minimal



In [ ]:
def prob_generator(fitness):

  min = np.min(fitness)
  fitness = fitness + np.absolute(min) + 1

  return fitness/np.sum(fitness)

**Roulette Wheel**
<br>
>implementing the roulette wheel is just a simple matter of using [np.random.choice](https://numpy.org/doc/stable/reference/random/generated/numpy.random.choice.html)

In [ ]:
def roulette_wheel(population,prob,pop_size):
  pop = np.random.choice(population,pop_size,p=prob)
  return pop 


###CROSS OVER
> we implement the two types of cross over 

1.   one point cross over
2.   uniform cross over
<br>
 both of which are implemented below 
<br>

*   note that we pass indiviual between binary and decimal whenever needed
*   the rest of the code is self-explanatory 




In [ ]:
def cross_over(type_,prob,parents,uniform_prob):
  if np.random.random_sample(1) < prob:
   if type_ == 1:
     return one_point_cross_over(prob,parents)
   elif type_ == 2:
      return uniform_cross_over(uniform_prob,parents)
  else:
    # print('no cross over! output type: ->',type(parents[0]))
    child1 = parents[0]
    child2 = parents[1]
    child1 = int(child1)
    child2 = int(child2)
    child1 = bin(child1)
    child2 = bin(child2)
    return child1[2:],child2[2:]

In [ ]:
def one_point_cross_over(prob,parents):
  
  father = int(parents[0])
  mother = int(parents[1])
  # print(type(father),'---------')
  # print(len(bin(father)))
  father = bin(int(parents[0]))[2:].zfill(5)
  mother = bin(int(parents[1]))[2:].zfill(5)
  
  # here we find the point at which cross-over happens 
  point = np.random.randint(1,len(father)-1,1)
  point[0] = point[0] + 2
  # print('point',point[0])
  # child1 = father[0:point[0]],mother[point[0]:] 
  # print(child1)
  child1 = father[0:point[0]]+mother[point[0]:] 
  # print('format',child1)
  child2 = mother[0:point[0]]+mother[point[0]:]
  return child1,child2

In [ ]:
def uniform_cross_over(uniform_prob,parents):
  father = parents[0]
  mother = parents[1]
  father = bin(father)[2:].zfill(5)
  mother = bin(mother)[2:].zfill(5)
  child1 = bin(0)[2:].zfill(5)
  child2 = bin(0)[2:].zfill(5)
  father = list(father)
  mother = list(mother)
  child1 = list(child1)
  child2 = list(child2)

  # using random.random_sample to make probability of each bits
  for i in range(len(father)):
    prob = np.random.random_sample(1)[0]
    # print('% is',prob,'round:',i)
    if prob < uniform_prob:
      child1[i] = father[i]
      child2[i] = mother[i]
    else:
      child1[i] = mother[i]
      child2[i] = father[i]

  return ''.join(child1),''.join(child2)

## Mutation 
> using the function *np.random.random_Sample* we generate the probability of each mutation and compare it to the given threshhold by the user 

In [ ]:
def mutation(prob,genome):
  # print('mutation input ------------->',genome)
  genome = list(genome)
  for i in range(len(genome)):
    temp = np.random.random_sample(1)
    # print('% is ',temp,' round:',i)
    if prob > temp:
      if genome[i] == 0:
        genome[i] = 1
      else: 
        genome[i] = 0
  genome = ''.join(map(str, genome))
  # print('mutated genome: -> ',genome)
  return genome


## Replacement Strategy 
> using the dataframe data-structure of pandas we **concat** the parents generation and new child generation together and sort it by **fitness** in an *deascending manner* and later on we take the best of the both generations 
<br>
<br>
**In other words we use fitness-base replacement strategy**

In [ ]:
def replacement(df_child,df_parent):
  temp = pd.concat([df_child, df_parent], axis=0)
  return temp.sort_values(by='fitness',ascending=False).reset_index(drop=True).loc[0:9]
  

#actual genetic algorithm

> implementation of the genetic algorithm optimizing \-x^2+6x for answers 
<br>
here we leverage the pandas dataframe structures to sort and organize each value of the indiviual 



In [ ]:
def ga(pop_size,mutation_rate,cross_rate,cross_type,termination):
  popu = intialize(pop_size)
  
  # print('-- going for the main loop')
  for i in tqdm(range(termination)):
    # print('initial population',popu)
    # print('\n generation',i,' :original population',popu)
    fitness = evaluation(popu)
    # print('\n fitness:\n',fitness)
    probability = prob_generator(fitness)
    # print('\n probability \n',probability)

    # expected_count = expected_number(10,popu)
    popu = roulette_wheel(popu,probability,pop_size)
    # print('generation',i,' :population after selection',popu)
    fitness = evaluation(popu)
    probability = prob_generator(fitness)


    # print('\n expected values \n',expected_count)
    selected_parents_df = pd.DataFrame(np.array([popu,fitness,probability]).T,columns=['population','fitness','probability'])
     
    # removes positional bias 
    childs_df =  selected_parents_df.sample(frac=1).reset_index(drop=True)
    for i in range (0,len(childs_df.index)-1,2):
      # print('---------test index',len(childs_df.index),' =>',i,i+1)
      child1 = childs_df.loc[i]['population']
      child2 = childs_df.loc[i+1]['population']
      # print('pair',i,'first child',child1)
      # print('pair',i,'second child',child2)
      child1 , child2  = cross_over(cross_type,cross_rate,(child1,child2),0.5)
      # print ('after cross ::::',child1,child2)
      child1 = mutation(mutation_rate,child1)
      child2 = mutation(mutation_rate,child2)
      child1 = int(child1,2)
      child2 = int(child2,2)
      # # print('after variations: ',child1,child2)
      childs_df.loc[i]['population'] = child1
      childs_df.loc[i+1]['population'] = child2
    # all variation end here we set on to calaculate the fitness and probability 
    childs_df['population'] = childs_df['population'].astype('int')
    childs_df['fitness'] = evaluation(np.array(childs_df['population'].values.tolist()))
    childs_df['probability'] = prob_generator(np.array(childs_df['fitness'].values.tolist()))
    # replacement strategy 
    replacement_Df =  replacement(childs_df,selected_parents_df)
    popu = np.array(replacement_Df['population'].values.tolist())
    # print('replaced popu:',popu)

  print()
  # return the best 10 of the last generation 
  return replacement_Df.head(10)






#INSERT INPUTS HERE

In [ ]:
population_size = 10 #@param {type:"integer"}
mutation_rate = 0.2 #@param {type:"number"}
cross_over_rate = 0.6 #@param {type:"number"}
cross_over_type = 1 #@param ["1", "2"] {type:"raw"}
termination_loop = 2 #@param {type:"integer"}
cross_over_type = int(cross_over_type)

ga(population_size,mutation_rate,cross_over_rate,cross_over_type,termination_loop)


100%|██████████| 2/2 [00:00<00:00, 36.31it/s]

,population,fitness,probability
0,3.0,9.0,0.110294
1,3.0,9.0,0.110294
2,3.0,9.0,0.100559
3,3.0,9.0,0.100559
4,3.0,9.0,0.100559
5,3.0,9.0,0.100559
6,3.0,9.0,0.100559
7,3.0,9.0,0.100559
8,3.0,9.0,0.100559
9,3.0,9.0,0.100559


**RESULTS OF THE LAST GENERATIONS ARE SHOWN ABOVE**